Ce notebook propose une analyse du Dataset 2a de la BCI Competition, qui comporte quatre classes : left hand (main gauche), right hand (main droite), tongue (langue) et feet (pieds). L'objectif est d'observer l'évolution des matrices de covariance des signaux à travers le temp.

<h1> A propos des données </h1>
Cet ensemble de données se compose de données EEG provenant de 9 sujets. Le paradigme BCI basé sur des indices se composait de quatre tâches d'imagerie motrice différentes, à savoir l'imagination du mouvement de la main gauche (classe 1), de la main droite (classe 2), des deux pieds (classe 3) et de la langue (classe 4). Deux sessions ont été enregistrées pour chaque sujet, à des jours différents. Chaque session est composée de 6 séquences séparées par de courtes pauses. Une séquence se compose de 48 essais (12 pour chacune des quatre classes possibles), ce qui donne un total de 288 essais par session.

Les sujets étaient assis dans un fauteuil confortable devant un écran d'ordinateur. Au début d'un essai (t = 0 s), une croix de fixation apparaissait sur l'écran noir. De plus, un bref signal sonore d'avertissement était présenté. Après deux secondes (t = 2 s), un indice sous forme d'une flèche pointant vers la gauche, la droite, le bas ou le haut (correspondant à l'une des quatre classes de la main gauche, de la main droite, du pied ou de la langue) apparaissait et restait sur l'écran pendant 1,25 s. Cela incitait les sujets à effectuer la tâche d'imagerie motrice désirée. Aucune rétroaction n'était fournie. Les sujets étaient invités à effectuer la tâche d'imagerie motrice jusqu'à ce que la croix de fixation disparaisse de l'écran à t = 6 s.

Vingt-deux électrodes Ag/AgCl (avec des distances inter-électrodes de 3,5 cm) ont été utilisées pour enregistrer l'EEG ; le montage est présenté dans la figure 3 à gauche. Tous les signaux ont été enregistrés de manière monopolaire, avec le mastoïde gauche comme référence et le mastoïde droit comme terre. Les signaux ont été échantillonnés à 250 Hz et filtrés en bande passante entre 0,5 Hz et 100 Hz. La sensibilité de l'amplificateur a été réglée à 100 μV. Un filtre coupe-bande supplémentaire à 50 Hz a été activé pour supprimer le bruit de ligne.

<h1>Importation et Prétraitement des Données :</h1> Les données de la première session du premier sujet sont chargées à partir du Dataset 2a. Les segments correspondant à chaque classe (left hand, right hand, tongue, feet) sont extraits de la  première session du premier run du premier sujet.

In [13]:
import mne
import numpy as np
from moabb.datasets import BNCI2014_001
import matplotlib.pyplot as plt
from pyriemann.utils.base import logm, expm
from scipy.linalg import sqrtm

def transport_vector(Delta, A, B):
    inv_B = np.linalg.inv(B)
    A_invB = np.dot(A, inv_B)
    E = sqrtm(A_invB)
    transported_Delta = np.dot(np.dot(E, Delta), E.T)
    return transported_Delta

def inverse_tangent_space_projection(ProjB_A, B):
    sqrt_B = sqrtm(B)
    sqrt_inv_B = np.linalg.inv(sqrt_B)
    transformed_ProjB_A = np.dot(np.dot(sqrt_inv_B, ProjB_A), sqrt_inv_B)
    exp_transformed_ProjB_A = expm(transformed_ProjB_A)
    original_A = np.dot(np.dot(sqrt_B, exp_transformed_ProjB_A), sqrt_B)
    return original_A

def tangent_space_projection(A, B):
    sqrt_B = sqrtm(B)
    sqrt_inv_B = np.linalg.inv(sqrt_B)
    transformed_A = np.dot(np.dot(sqrt_inv_B, A), sqrt_inv_B)
    log_transformed_A = logm(transformed_A)
    tangent_space_A = np.dot(np.dot(sqrt_B, log_transformed_A), sqrt_B)
    return tangent_space_A

def compute_trajectoires(dataset,subject,session_name,run_name, tmin, tmax, tstride, stride):
    
    sessions = dataset.get_data(subjects=[subject])
    raw = sessions[subject][session_name][run_name]
    events, event_id = mne.events_from_annotations(raw)
    epochs = mne.Epochs(raw, events, event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True)
    
    cov_list = []
    projections = []
    label_list = []
    trajectoire = []
    label =[]

    for i in range(epochs.events.shape[0]):
        epoch = epochs[i]
        start = 0
        end = tstride
        cpt = 0
        cropped_epoch = epoch.copy().crop(tmin=0, tmax=tstride)
        cov_segment = mne.compute_covariance(cropped_epoch, method='empirical')
        cov_list.append(cov_segment['data'])
        label_list.append(epoch.events[0, -1])
        while end <= tmax - tmin:
            start += stride
            end = start + tstride
            cropped_epoch = epoch.copy().crop(tmin=start, tmax=end)
            if cropped_epoch.get_data().shape[-1] > 1:
                cov_segment = mne.compute_covariance(cropped_epoch, method='empirical')
                cov_list.append(cov_segment['data'])
                label_list.append(epoch.events[0, -1])
                projections.append(tangent_space_projection(cov_list[i+cpt], cov_list[i+cpt+1]))
                for j in range(i+cpt-1, -1, -1):
                    projections[j] = transport_vector(projections[j], cov_list[i+cpt+1], cov_list[i+cpt])
            cpt += 1
        cpt_sum = 0
        for k in range(i, i+cpt):
            cpt_sum += projections[k]
            if (k + 1) % cpt == 0:  # Si le nombre de matrices de covariance atteint cpt
                trajectoire.append(cpt_sum)
                label.append(label_list[i]%cpt) 
                cpt_sum = 0


    return cov_list,projections, label_list,  trajectoire , label



In [14]:
# Exemple d'utilisation de la méthode
dataset = BNCI2014_001()
subject = 1
session_name = "0train"
run_name = "0"
tmin = -1
tmax = 4
tstride = 2
stride = 0.5

cov_list, projections, label_list, trajectoire, label = compute_trajectoires(dataset,subject,session_name,run_name, tmin, tmax, tstride, stride)

# Afficher la matrice résultante
print(f"Nombre de covariances calculées : {len(cov_list)}")
print(f"Labels associés : {label_list}")


Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Not setting metadata
48 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 48 events and 1251 original time points ...
0 bad epochs dropped
Computing rank from data with rank=None
    Using tolerance 5.2e-12 (2.2e-16 eps * 22 dim * 1.1e+03  max singular value)
    Estimated rank (eeg): 22
    EEG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Number of samples used : 501
[done]


C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214.py:50: RuntimeWarning: Epochs are not baseline corrected, covariance matrix may be inaccurate
  cov_segment = mne.compute_covariance(cropped_epoch, method='empirical')
C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214.py:57: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  if cropped_epoch.get_data().shape[-1] > 1:
C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214.py:57: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  if cropped_epoch.get_data().shape[-1] > 1:
C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214.py:57: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  if cropped_epoch.get_data().shape[-1] > 1:
C:\Users\ABC\AppData\Local\Temp\i

Computing rank from data with rank=None
    Using tolerance 4.9e-12 (2.2e-16 eps * 22 dim * 1e+03  max singular value)
    Estimated rank (eeg): 22
    EEG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Number of samples used : 501
[done]


C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214.py:56: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.Epochs'>.tmax (4 s)
  cropped_epoch = epoch.copy().crop(tmin=start, tmax=end)
C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214.py:57: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  if cropped_epoch.get_data().shape[-1] > 1:
C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214.py:50: RuntimeWarning: Epochs are not baseline corrected, covariance matrix may be inaccurate
  cov_segment = mne.compute_covariance(cropped_epoch, method='empirical')
C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214.py:57: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  if cropped_epoch.get_data().shape[-1] > 1:
C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214

Computing rank from data with rank=None
    Using tolerance 5.1e-12 (2.2e-16 eps * 22 dim * 1e+03  max singular value)
    Estimated rank (eeg): 22
    EEG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Number of samples used : 501
[done]


...

Computing rank from data with rank=None
    Using tolerance 5.1e-12 (2.2e-16 eps * 22 dim * 1e+03  max singular value)
    Estimated rank (eeg): 22
    EEG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Number of samples used : 501
[done]


C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214.py:56: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.Epochs'>.tmax (4 s)
  cropped_epoch = epoch.copy().crop(tmin=start, tmax=end)
C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214.py:57: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  if cropped_epoch.get_data().shape[-1] > 1:
C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214.py:50: RuntimeWarning: Epochs are not baseline corrected, covariance matrix may be inaccurate
  cov_segment = mne.compute_covariance(cropped_epoch, method='empirical')
C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214.py:57: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  if cropped_epoch.get_data().shape[-1] > 1:
C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214

Computing rank from data with rank=None
    Using tolerance 4.7e-12 (2.2e-16 eps * 22 dim * 9.7e+02  max singular value)
    Estimated rank (eeg): 22
    EEG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Number of samples used : 501
[done]


C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214.py:50: RuntimeWarning: Epochs are not baseline corrected, covariance matrix may be inaccurate
  cov_segment = mne.compute_covariance(cropped_epoch, method='empirical')
C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214.py:57: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  if cropped_epoch.get_data().shape[-1] > 1:
C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214.py:57: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  if cropped_epoch.get_data().shape[-1] > 1:
C:\Users\ABC\AppData\Local\Temp\ipykernel_12960\910821214.py:57: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  if cropped_epoch.get_data().shape[-1] > 1:
C:\Users\ABC\AppData\Local\Temp\i

Nombre de covariances calculées : 384
Labels associés : [4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4